# Syft RPC Application Template Tutorial

## Introduction

This tutorial demonstrates how to use the Syft RPC Application Template to build distributed applications with bidirectional communication between Syft datasites. The template provides a foundation for creating applications that can:

1. **Send requests** to other Syft datasites
2. **Receive and process requests** from other datasites
3. **Discover available datasites** and identify which ones have compatible services

We'll explore two use cases:

1. **PingPong**: A simple example included with the template that demonstrates basic request-response functionality
2. **Custom Application**: How to build your own application by extending the template

### How It Works

The template combines server and client functionality:

- **Server Component**: Runs in a background thread to listen for and respond to incoming requests
- **Client Component**: Sends requests to other datasites and processes responses

This dual-mode operation allows each instance to both provide and consume services.

## 1. Setup

First, let's import the PingPong library which contains our RPC application template:

In [1]:
import pingpong as pp

### Environment Prerequisites

This notebook assumes you have set up Syft with at least one datasite. For a complete testing environment, you'll want to have two or more SyftBox clients running with different accounts.

For example:
- Alice's SyftBox on port 8082
- Bob's SyftBox on port 8081

See the README.md for instructions on setting up these environments.

## 2. Using the PingPong Example

Let's start with the included PingPong example, which demonstrates the basic functionality of the template.

### 2.1 Creating PingPong Clients

We'll create clients connected to our SyftBox instances. Each client will start a server in the background.

In [2]:
# Create a client for Bob's account
bob_client = pp.client("~/.syft_bob_config.json")

2025-03-08 19:30:00.973 | INFO     | pingpong:__init__:67 - 🔑 Connected as: bob@openmined.org
2025-03-08 19:30:00.973 | INFO     | pingpong:_run_server:85 - 🚀 SERVER: Running pingpong server as bob@openmined.org
2025-03-08 19:30:00.974 | INFO     | pingpong:_start_server:80 - 🔔 Server started for bob@openmined.org
2025-03-08 19:30:00.976 | INFO     | syft_event.server2:register_rpc:140 - Register RPC: /ping
2025-03-08 19:30:00.976 | INFO     | pingpong:_run_server:109 - 📡 SERVER: Listening for requests at /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/pingpong/rpc
2025-03-08 19:30:00.978 | INFO     | syft_event.server2:publish_schema:96 - Published schema to /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/pingpong/rpc/rpc.schema.json
2025-03-08 19:30:18.826 | DEBUG    | syft_event.server2:process_pending_requests:105 - Processing pending request bb2e1f93-5375-4b55-915c-b7caa7996875.request
2025-03-08 19:30:18.830 | INFO     | pingpong:_handl

In [3]:
# Create a client for Alice's account 
# Comment this out if you only have one Syft instance available
alice_client = pp.client("~/.syft_alice_config.json")

2025-03-08 19:30:02.058 | INFO     | pingpong:__init__:67 - 🔑 Connected as: alice@openmined.org
2025-03-08 19:30:02.059 | INFO     | pingpong:_run_server:85 - 🚀 SERVER: Running pingpong server as alice@openmined.org
2025-03-08 19:30:02.060 | INFO     | pingpong:_start_server:80 - 🔔 Server started for alice@openmined.org
2025-03-08 19:30:02.062 | INFO     | syft_event.server2:register_rpc:140 - Register RPC: /ping
2025-03-08 19:30:02.065 | INFO     | pingpong:_run_server:109 - 📡 SERVER: Listening for requests at /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/api_data/pingpong/rpc
2025-03-08 19:30:02.071 | INFO     | syft_event.server2:publish_schema:96 - Published schema to /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/api_data/pingpong/rpc/rpc.schema.json


### 2.2 Discovering Datasites

Now let's see what datasites are available. The template provides two methods for this:

1. `list_datasites()`: Lists all available datasites
2. `list_available_servers()`: Lists only datasites with the application server running

In [4]:
# Get all available datasites
all_datasites = bob_client.list_datasites()

# Print the number of datasites and first few examples
print(f"Total datasites available: {len(all_datasites)}")
print("Sample datasites:")
for ds in all_datasites[:5]:  # Show just the first 5
    print(f"  - {ds}")

Total datasites available: 140
Sample datasites:
  - Morganabuell98@gmail.com
  - a@gmail.com
  - a@openmined.org
  - abinvarghese90@gmail.com
  - alice@openmined.org


In [5]:
# Get only datasites with the PingPong server running
active_servers = bob_client.list_available_servers()

print(f"Active PingPong servers: {len(active_servers)}")
print("Available servers:")
for server in active_servers:
    print(f"  - {server}")

Active PingPong servers: 5
Available servers:
  - alice@openmined.org
  - bob@openmined.org
  - khoa@openmined.org
  - rasswanth@openmined.org
  - yash@openmined.org


### 2.3 Sending a Ping

Now let's send a ping from Bob's client to Alice's client. This demonstrates the basic request-response pattern.

In [6]:
# If Alice's client is in the active servers list, ping it
if 'alice@openmined.org' in active_servers:
    response = bob_client.ping('alice@openmined.org')
    print(f"Response from Alice: {response.msg}")
    print(f"Timestamp: {response.ts}")
else:
    # Find the first available server
    if active_servers:
        target = active_servers[0]
        print(f"Alice not found, pinging {target} instead")
        response = bob_client.ping(target)
        print(f"Response: {response.msg}")
    else:
        print("No active servers found to ping")

2025-03-08 19:30:05.185 | INFO     | pingpong:send_request:168 - 📤 SENDING: Request to alice@openmined.org
2025-03-08 19:30:08.186 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/api_data/pingpong/rpc/ping/04717fc6-99f2-4183-9bbe-61ad2a5685c4.request
2025-03-08 19:30:08.187 | INFO     | pingpong:_handle_request:244 - 🔔 RECEIVED: Ping request - msg='Hello from bob@openmined.org!' ts=datetime.datetime(2025, 3, 9, 0, 30, 5, 185771, tzinfo=TzInfo(UTC))
2025-03-08 19:30:12.138 | INFO     | pingpong:send_request:183 - 📥 RECEIVED: Response from alice@openmined.org. Time: 6.95s


Response from Alice: Pong from alice@openmined.org
Timestamp: 2025-03-09 00:30:08.187784+00:00


### 2.4 Bidirectional Communication

If we have both Alice and Bob's clients running, we can demonstrate bidirectional communication:

In [7]:
# This section assumes you have both clients running
# Alice pings Bob
try:
    alice_response = alice_client.ping('bob@openmined.org')
    print(f"Alice received: {alice_response.msg}")
except NameError:
    print("Alice's client not defined. Skipping bidirectional test.")

2025-03-08 19:30:15.080 | INFO     | pingpong:send_request:168 - 📤 SENDING: Request to bob@openmined.org
2025-03-08 19:30:18.846 | DEBUG    | syft_event.handlers:on_any_event:31 - FSEvent - created - /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/api_data/pingpong/rpc/ping/bb2e1f93-5375-4b55-915c-b7caa7996875.request
2025-03-08 19:30:18.850 | INFO     | pingpong:_handle_request:244 - 🔔 RECEIVED: Ping request - msg='Hello from alice@openmined.org!' ts=datetime.datetime(2025, 3, 9, 0, 30, 15, 80859, tzinfo=TzInfo(UTC))
2025-03-08 19:30:22.674 | INFO     | pingpong:send_request:183 - 📥 RECEIVED: Response from bob@openmined.org. Time: 7.59s


Alice received: Pong from bob@openmined.org


### 2.5 Error Handling

The template includes robust error handling. Let's see what happens when we ping a non-existent datasite:

In [8]:
# Try to ping a datasite that doesn't exist
error_response = bob_client.ping("nonexistent@example.com")
print(f"Response when pinging invalid datasite: {error_response}")

2025-03-08 19:30:24.091 | ERROR    | pingpong:send_request:155 - Invalid datasite: nonexistent@example.com
2025-03-08 19:30:24.091 | INFO     | pingpong:send_request:156 - Available datasites:
2025-03-08 19:30:24.092 | INFO     | pingpong:send_request:158 -   - Morganabuell98@gmail.com
2025-03-08 19:30:24.092 | INFO     | pingpong:send_request:158 -   - a@gmail.com
2025-03-08 19:30:24.093 | INFO     | pingpong:send_request:158 -   - a@openmined.org
2025-03-08 19:30:24.093 | INFO     | pingpong:send_request:158 -   - abinvarghese90@gmail.com
2025-03-08 19:30:24.093 | INFO     | pingpong:send_request:158 -   - alice@openmined.org
2025-03-08 19:30:24.093 | INFO     | pingpong:send_request:158 -   - amanpriyanshusms2001@gmail.com
2025-03-08 19:30:24.093 | INFO     | pingpong:send_request:158 -   - amtulifra12@gmail.com
2025-03-08 19:30:24.093 | INFO     | pingpong:send_request:158 -   - andrej@jovanovic.co.za
2025-03-08 19:30:24.094 | INFO     | pingpong:send_request:158 -   - andrew@openm

Response when pinging invalid datasite: None


## 3. Creating Your Own RPC Application

Now let's explore how to create your own application by extending the template. We'll create a simple Weather Service that provides weather forecasts.

### 3.1 Define Your Models

First, we need to define the request and response models for our application:

In [ ]:
from pingpong import SyftRPCClient
from pydantic import BaseModel, Field
from datetime import datetime
import random

# Define request model
class WeatherRequest(BaseModel):
    location: str = Field(description="City or coordinates")
    units: str = Field(default="metric", description="Temperature units (metric/imperial)")

# Define response model
class WeatherResponse(BaseModel):
    location: str = Field(description="Location of forecast")
    temperature: float = Field(description="Current temperature")
    conditions: str = Field(description="Weather conditions")
    timestamp: datetime = Field(description="Forecast time")

### 3.2 Create Your Client Class

Next, we'll create our Weather client by extending the SyftRPCClient base class:

In [ ]:
class WeatherClient(SyftRPCClient):
    def __init__(self, config_path=None):
        """Initialize the Weather client."""
        super().__init__(
            config_path=config_path,
            app_name="weather",                  # Custom app name
            endpoint="/forecast",                # Custom endpoint
            request_model=WeatherRequest,        # Our request model
            response_model=WeatherResponse       # Our response model
        )
    
    def _handle_request(self, request_data, ctx, box):
        """Handle incoming weather requests."""
        # In a real app, you'd look up actual weather data
        # This is just a mock implementation
        conditions = ["Sunny", "Cloudy", "Rainy", "Windy", "Snowy"]
        
        # Generate random temperature based on units
        if request_data.units == "metric":
            temp = round(random.uniform(15.0, 30.0), 1)
        else:  # imperial
            temp = round(random.uniform(60.0, 90.0), 1)
        
        return WeatherResponse(
            location=request_data.location,
            temperature=temp,
            conditions=random.choice(conditions),
            timestamp=datetime.now()
        )
    
    def get_forecast(self, email, location, units="metric"):
        """Request a weather forecast from another datasite."""
        request = WeatherRequest(location=location, units=units)
        return self.send_request(email, request)

### 3.3 Testing Our Custom Application

Now let's create instances of our Weather client and test it:

In [ ]:
# Create weather clients
bob_weather = WeatherClient("~/.syft_bob_config.json")

try:
    alice_weather = WeatherClient("~/.syft_alice_config.json")
    have_alice = True
except Exception:
    have_alice = False

In [ ]:
# Check which datasites are running our weather service
weather_servers = bob_weather.list_available_servers()
print(f"Active Weather Services: {len(weather_servers)}")
for server in weather_servers:
    print(f"  - {server}")

In [ ]:
# Get weather forecast if possible
if 'alice@openmined.org' in weather_servers and have_alice:
    # Bob requests weather from Alice
    nyc_forecast = bob_weather.get_forecast('alice@openmined.org', "New York", "metric")
    print(f"Weather in {nyc_forecast.location}: {nyc_forecast.temperature}°C, {nyc_forecast.conditions}")
    
    # Alice requests weather from Bob (for London in imperial units)
    london_forecast = alice_weather.get_forecast('bob@openmined.org', "London", "imperial")
    print(f"Weather in {london_forecast.location}: {london_forecast.temperature}°F, {london_forecast.conditions}")
else:
    print("No weather servers available or Alice client not running")

### 3.4 Advanced: Creating Multiple Endpoints

In a real application, you might want to support multiple endpoints. While beyond the scope of this tutorial, you could extend the base class to register multiple endpoints in the `_run_server` method.

## 4. Cleanup

When we're done, it's important to properly close all clients to shut down the background servers:

In [ ]:
# Close PingPong clients
bob_client.close()
try:
    alice_client.close()
except NameError:
    pass  # Alice client wasn't created

In [ ]:
# Close Weather clients
bob_weather.close()
if have_alice:
    alice_weather.close()

## 5. Conclusion

In this tutorial, we've demonstrated:

1. Using the included PingPong application to send and receive pings
2. Discovering available datasites and servers
3. Creating a custom Weather application by extending the base class
4. Handling errors and properly cleaning up resources

The Syft RPC Application Template provides a powerful foundation for building distributed applications on the Syft platform. By extending the base class and defining your own models, you can create sophisticated applications with minimal boilerplate code.

### Next Steps

- Add authentication to your application endpoints
- Create more complex data models for your domain
- Implement real-world business logic in your request handlers
- Build applications that integrate with external APIs or databases
- Create applications with multiple endpoints for different functions